# Principal component analysis

## Libraries

In [2]:
import numpy as np
import pandas as pd
    
from sklearn.decomposition import PCA
from sklearn import preprocessing

import plotly.express as px
import plotly.graph_objects as go

## Dataset

In [3]:
train_features = pd.read_csv("Data/train_features.csv")
test_features = pd.read_csv("Data/test_features.csv")
train_targets = pd.read_csv("Data/train_targets_scored.csv")

In [4]:
# Separate data for PCA (Gene expression (ge) and cell viability (cv) data)
train_ge = train_features.iloc[:,4:776]
train_cv = train_features.iloc[:,776:876]
test_ge = test_features.iloc[:,4:776] 
test_cv = test_features.iloc[:,776:876] 

In [5]:
# Add sum of MoA column to train_targets
train_targets['moa_sum'] = "0"

for i in range(len(train_targets)):
    moa_sum = train_targets.iloc[i,1:207].sum()
    if moa_sum == 1:
        train_targets["moa_sum"][i] = "1"
    if moa_sum > 1:
        train_targets["moa_sum"][i] = "> 2"

ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## PCA for gene expressions

In [6]:
# Calculate principal components
pca_ge = PCA()
pca_ge.fit(train_ge)
pca_ge_data = pca_ge.transform(train_ge)

# Determine the percentage of variance explained by each PC
pca_ge_var = np.round(pca_ge.explained_variance_ratio_*100, decimals = 1)

In [14]:
# Plot supporting variables
labels_ge = ['PC' + str(x) for x in range (1, len(pca_ge_var) +1)]
pca_ge_cumul = pca_ge_var.cumsum()
pca_ge_sum = [sum(pca_ge_var)]* len(pca_ge_var)
upper_range = 301

# Create figure
fig = go.Figure()

# Scree plot (bar trace)
fig.add_trace(
    go.Bar(
        x = labels_ge[:upper_range], 
        y = pca_ge_var[:upper_range], 
        name = "Variation explained by each PC"))

# Cumulative plot (Line trace)
fig.add_trace(
    go.Scatter(
        x = labels_ge[:upper_range], 
        y = pca_ge_cumul[:upper_range], 
        mode='lines', 
        name = "Variation explained by sum of all preceeding PC"))

# Maximum plot (Line trace of maximum explained variation)
fig.add_trace(
    go.Scatter(
        x = labels_ge[:upper_range], 
        y = pca_ge_sum[:upper_range], 
        mode='lines', 
        line_color = '#555555', 
        name = "Variation explained by all PC", 
        opacity = 0.5))

# Layout
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    margin={'l': 20, 'r': 20, 't':  20, 'b': 20},
    template="simple_white",
    legend_x = 0.5,
    legend_y = 0.05,
    xaxis_title = "Principal component number",
    yaxis_title = "Variation explained (%)", 
    xaxis_dtick = 50,
    yaxis_range = [0,100])
    
#HTML export
fig.write_html(file="PCA_scree_ge.html",include_plotlyjs="directory",full_html=True)

In [15]:
# Merge PC and MoA sum data for all datapoints
pca_ge_moa = pd.merge(pd.DataFrame(pca_ge_data, columns = labels_ge), train_targets['moa_sum'], how = 'inner', left_index = True, right_index = True)

# Scatterplot matrix for top 5 PC
fig = px.scatter_matrix(
    pca_ge_moa, 
    dimensions = ["PC1", "PC2", "PC3", "PC4", "PC5"],
    color = "moa_sum",
    opacity = 0.2, 
    template = "simple_white")


fig.update_traces(marker_size = 3)

# Layout
fig.update_layout(
    autosize = False,
    width = 800,
    height = 800,
    margin = {'l': 20, 'r': 20, 't':  20, 'b': 20})

# HTML export
fig.write_html(file="Scatterplot_PCA_GE.html",include_plotlyjs="directory",full_html=True)

## PCA - cell viability

In [8]:
# Calculate PC
pca_cv = PCA()
pca_cv.fit(train_cv)
pca_cv_data = pca_cv.transform(train_cv)

# Determine the percentage of variance explained by each PC
pca_cv_var = np.round(pca_cv.explained_variance_ratio_*100, decimals = 1)

In [9]:
# Plot supporting variables
labels_cv = ['PC' + str(x) for x in range (1, len(pca_cv_var) +1)]
pca_cv_cumul = pca_cv_var.cumsum()
pca_cv_sum = [sum(pca_cv_var)]* len(pca_cv_var)
upper_range = 100

# Create figure
fig = go.Figure()

# Scree plot (bar trace)
fig.add_trace(
    go.Bar(
        x = labels_cv[:upper_range], 
        y = pca_cv_var[:upper_range], 
        name = "Variation explained by each PC"))

# Cumulative plot (Line trace)
fig.add_trace(
    go.Scatter(
        x = labels_cv[:upper_range], 
        y = pca_cv_cumul[:upper_range], 
        mode='lines', 
        name = "Variation explained by sum of all preceeding PC"))

# Maximum plot (Line trace of maximum explained variation)
fig.add_trace(
    go.Scatter(
        x = labels_cv[:upper_range], 
        y = pca_cv_sum[:upper_range], 
        mode='lines', 
        line_color = '#555555', 
        name = "Variation explained by all PC", 
        opacity = 0.5))

# Layout
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    margin={'l': 20, 'r': 20, 't':  20, 'b': 20},
    template="simple_white",
    legend_x = 0.5,
    legend_y = 0.05,
    xaxis_title = "Principal component number",
    yaxis_title = "Variation explained (%)", 
    xaxis_dtick = 20,
    yaxis_range = [0,100])
    
#HTML export
fig.write_html(file="PCA_scree_cv.html",include_plotlyjs="directory",full_html=True)

In [10]:
# Merge PC and MoA sum data for all datapoints
pca_cv_moa = pd.merge(pd.DataFrame(pca_cv_data, columns = labels_cv), train_targets['moa_sum'], how = 'inner', left_index = True, right_index = True)

# Scatterplot matrix for top 5 PC
fig = px.scatter_matrix(
    pca_cv_moa, 
    dimensions = ["PC1", "PC2", "PC3", "PC4", "PC5"],
    color = "moa_sum",
    opacity = 0.3, 
    template = "simple_white")


fig.update_traces(marker_size = 4)

# Layout
fig.update_layout(
    autosize = False,
    width = 800,
    height = 800,
    margin = {'l': 20, 'r': 20, 't':  20, 'b': 20})

# HTML export
fig.write_html(file="Scatterplot_PCA_CV.html",include_plotlyjs="directory",full_html=True)

## Cross-correlation between gene expression features and cell viability PC features

In [11]:
# Merge gene expression and cell viability PCA data for all datapoints
ge_pca_cv = pd.merge(train_ge, pd.DataFrame(pca_cv_data, columns = labels_cv).iloc[:,:97], how = 'inner', left_index = True, right_index = True)

# Obtianing a dataframe of the correlations between gene expression and cell viability PCA features
corr = ge_pca_cv.corr()
ge_pca_cv_corr = corr.iloc[772:870,0:772]
ge_pca_cv_corr

# List of correlations
ge_pca_cv_correlations_list = []

for i in range(len(ge_pca_cv_corr)):
    for j in list(ge_pca_cv_corr.iloc[i,]): 
        ge_pca_cv_correlations_list.append(j)

In [12]:
# Calculating the number of correlations above with a correlation coefficient (CC) above 0.2, 0.4, 0.6 and 0.8
ge_pca_cv_correlations_2 = list(filter(lambda a: a < 0.2 and a > -0.2, ge_pca_cv_correlations_list)) 
ge_pca_cv_correlations_4 = list(filter(lambda a: a < 0.4 and a > -0.4, ge_pca_cv_correlations_list)) 
ge_pca_cv_correlations_6 = list(filter(lambda a: a < 0.6 and a > -0.6, ge_pca_cv_correlations_list)) 
ge_pca_cv_correlations_8 = list(filter(lambda a: a < 0.8 and a > -0.8, ge_pca_cv_correlations_list)) 


print(f"Number of correlations (CC < 0.2): {len(ge_pca_cv_correlations_2)}")
print(f"Percent of correlations (CC < 0.2): {round(len(ge_pca_cv_correlations_2)/len(ge_pca_cv_correlations_list)*100, 1)}")

print(f"Number of correlations (0.2 < CC < 0.4): {len(ge_pca_cv_correlations_4) - len(ge_pca_cv_correlations_2)}")
print(f"Percent of correlations (0.2 < CC < 0.4): {round((len(ge_pca_cv_correlations_4)- len(ge_pca_cv_correlations_2))/len(ge_pca_cv_correlations_list)*100, 1)}")

print(f"Number of correlations (0.4 < CC < 0.6): {len(ge_pca_cv_correlations_6) - len(ge_pca_cv_correlations_4)}")
print(f"Percent of correlations (0.4 < CC < 0.6): {round((len(ge_pca_cv_correlations_6)- len(ge_pca_cv_correlations_4))/len(ge_pca_cv_correlations_list)*100, 1)}")

print(f"Number of correlations (0.6 < CC < 0.8): {len(ge_pca_cv_correlations_8) - len(ge_pca_cv_correlations_6)}")
print(f"Percent of correlations (0.6 < CC < 0.8): {round((len(ge_pca_cv_correlations_8)- len(ge_pca_cv_correlations_6))/len(ge_pca_cv_correlations_list)*100, 1)}")

Number of correlations (CC < 0.2): 74038
Percent of correlations (CC < 0.2): 98.9
Number of correlations (0.2 < CC < 0.4): 470
Percent of correlations (0.2 < CC < 0.4): 0.6
Number of correlations (0.4 < CC < 0.6): 208
Percent of correlations (0.4 < CC < 0.6): 0.3
Number of correlations (0.6 < CC < 0.8): 149
Percent of correlations (0.6 < CC < 0.8): 0.2


In [13]:
fig = go.Figure() 

# Histogram trace
fig.add_trace(
    go.Histogram(
        x=ge_pca_cv_correlations_list, 
        nbinsx = 150))

# Layout
fig.update_layout(autosize=False,
    width=400,
    height=300,
    showlegend=False,     
    margin={'l': 20, 'r': 20, 't':  20, 'b': 20},
    template="simple_white",
    xaxis_title = "Correlation coefficient",
    yaxis_title = "Count",
    xaxis_range = [-1,1])
    
#HTML export
fig.write_html(file="ge_pca_cv_correlations.html",include_plotlyjs="directory",full_html=True)